In [1]:
# Configure Environment 
from  dev.data.settings import * 
print("Platform: " , PLATFORM)
print("Data directory : ", os.getcwd())



Platform:  dev
Data directory :  /home/jovyan/dev/data


In [2]:
import os
import sys
import time
import logging
import numpy as np
import pandas as pd
from datetime import datetime
import urllib.parse
import http.client
import json
import pickle
import logging
logger = logging.getLogger('personCasting')
logger.setLevel(logging.INFO)

In [3]:
import numpy as np
import nltk
from nltk.stem import PorterStemmer

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stopwords=  set(stopwords.words('english'))

from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Extract Movies Plots for each Celebrity by genre.  
* Combine all plots across movies into one text


In [4]:
path_to_json = './cast/themoviedb/'
json_files = [path_to_json+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') ]
len(json_files)

theMovieDB=[]
for fname in json_files:
    try:
        with open(fname, 'r') as f:
            r = json.load(f)   
            for row in r:
                theMovieDB.append(row)

    except Exception as e:
        #extract.update({ columns[key]  : '' } )
        logger.info('Unable to find key : '+ fname + '::' +str(e))        
        

In [6]:
theMovieDB=[]
for fname in json_files:
    try:
        with open(fname, 'r') as f:
            r = json.load(f)   
            for row in r:
                theMovieDB.append(row)

    except Exception as e:
        #extract.update({ columns[key]  : '' } )
        logger.info('Unable to find key : '+ fname + '::' +str(e))        
        
castDF =  pd.DataFrame.from_dict(theMovieDB , dtype=str)
castDF.head()        

In [9]:
path_to_json = './movies/wikipedia/'
json_files = [path_to_json+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') and os.stat(path_to_json+pos_json).st_size>0]
len(json_files)

moviesPlot=[]

for fname in json_files:
    with open(fname, 'r') as f:
        r = json.load(f)   
        columns= {'_movieID': '_movieID', 'plot': 'plot', 'summary': 'summary' }
        
        extract ={}
        for key in columns:
            try: 
                extract.update({ columns[key] : r[key] } )
            except Exception as e:
                extract.update({ columns[key]  : '' } )
                logger.info('Unable to find key : '+ key + '::' +str(e))

        moviesPlot.append(extract)
    
plotDF =  pd.DataFrame.from_dict(moviesPlot , dtype=str)
plotDF[plotDF['plot']==''].head()  

In [12]:
path_to_json = './movies/imdb/'
json_files = [path_to_json+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') and os.stat(path_to_json+pos_json).st_size>0]
len(json_files)

movieGenre =[]

for fname in json_files:
    with open(fname, 'r') as f:
        r = json.load(f)   
        columns= {'_id': '_movieID', 'plot outline': 'plot', 'genres': 'genres' }
        
        extract ={}
        for key in columns:
            try: 
                if key == 'genres':
                    extract.update({'genres': ','.join(r[key]) })
                else:
                    extract.update({ columns[key] : r[key] } )
            except Exception as e:
                extract.update({ columns[key]  : '' } )
                logger.info('Unable to find key : '+ key + '::' +str(e))

        movieGenre.append(extract)
    
movieGenreDF =  pd.DataFrame.from_dict(movieGenre , dtype=str)
movieGenreDF.head()   

In [16]:
castDF['movieID'] = castDF['movieID'].astype(str)
castMoivesDF = castDF.merge( plotDF, how='inner', left_on = 'movieID', right_on = '_movieID')
castMoivesDF = castMoivesDF.merge( movieGenreDF, how='inner', left_on = '_movieID', right_on = '_movieID')

castMoivesDF['plotCombine'] = castMoivesDF['movieTitle']+ ' '+ castMoivesDF['plot_x']+' '+castMoivesDF['summary']+' '+castMoivesDF['plot_y']
extractDF=castMoivesDF[['_actorID', 'actor', '_movieID', 'plotCombine', 'genres']]


# Combine all the movie plots from different data sources into one field
# Given a moive can belong into multiple genre, create row for each movie plot

extractDF = extractDF['genres'].str.split(',').apply(pd.Series) \
.merge(extractDF, right_index = True, left_index=True) \
.drop(['genres'], axis =1) \
.melt(id_vars =['_actorID', 'actor', '_movieID', 'plotCombine'], value_name='genre') \
.drop('variable', axis=1) \
.dropna()

# Combine all the different movie plots by actor and genre.

actorGenreSummary = extractDF.groupby(['_actorID', 'actor', 'genre'])['plotCombine'].apply(lambda x: ' '.join(x)).reset_index()

# Get counts by genre
extractDF.groupby('genre').count()

,_actorID,actor,_movieID,plotCombine,genres
0,5,Bradley Cooper,28355,Case 39 Emily Jenkins (Renée Zellweger) is a s...,"Horror,Mystery,Thriller"
1,5,Bradley Cooper,45243,The Hangover Part II Two years after the event...,"Comedy,Mystery"
2,5,Bradley Cooper,18785,The Hangover To celebrate his upcoming marriag...,Comedy
3,5,Bradley Cooper,97367,"The Place Beyond the Pines In 1997, Luke Glant...","Crime,Drama,Thriller"
4,5,Bradley Cooper,109439,The Hangover Part III Two years after the even...,"Adventure,Comedy,Crime"


In [21]:
# Get a list of generes

', '.join(extractDF['genre'].unique().tolist())

'Horror, Comedy, Crime, Adventure, Action, Drama, Documentary, Short, Animation, Music, Biography, Sport, Family, , Fantasy, Romance, Game-Show, Musical, Mystery, Thriller, Reality-TV, Sci-Fi, History, War, Western, News, Film-Noir'

In [22]:
actorGenreCatalog = actorGenreSummary.to_json(orient='records')
actorGenreJson = eval(actorGenreCatalog)

# Save model lookup 
pickle.dump( actorGenreJson, open( "./models/actorGenreDict.pkl", "wb" ) )

1228

# Clean final Plot Text 

In [25]:
def tokenizeDoc(docs, stopwords=stopwords ):
    i=0
    for doc in docs:
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(doc.lower())
        # Stem words
        #ps = PorterStemmer()
        #tokens = [ps.stem(word) for word in tokens]
        # Remove stop words.
        tokens = [word for word in tokens if word not in stopwords]
        fdist1 = nltk.FreqDist(tokens)
        words = list(word for word, freq in fdist1.items() if not word.isdigit())

        docs[i] = ' '.join(words[:3000])
        i += 1
    return docs

In [26]:
def cleanSentences(sentences):
    punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'  #TBD
        #TBD - find sentence splitter - 
        # remove punctuation, strip white space; remove non-standard characters; remove numbers; 
    i = 0
    for s in sentences:
        s = s.strip()
        s = s.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ').replace("[0-9]", " ")
        s = s.replace(',', '').replace('.','') #TBD - replace with punctuation
        #s = s[0:2000]
        sentences[i] = s
        i += 1
    return sentences

In [27]:
sentencesAll = actorGenreSummary['plotCombine'].tolist()
len(sentencesAll) 
sentences = tokenizeDoc(sentencesAll)
sentences[1]

# Start ELMo Word Embedding Modeling

In [29]:
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub

config = tf.ConfigProto()
config.intra_op_parallelism_threads = 200
config.gpu_options.allow_growth = True
tf.Session(config=config)


url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

elmodir = os.getcwd()+'/models/'

In [30]:
def embed_text(text):
    vectors = session.run(embeddings, feed_dict={text_ph: text})
    return [vector.tolist() for vector in vectors]

In [31]:
# score n Sentences at time - to pipeline this
def defineEmbeddings (start, end):    
    embeddings = embed(
        sentences[start:end],
        signature="default",
        as_dict=True)["default"]
    return embeddings# Save Weights - use names based on Movie 

import pickle as pkl
# score sentences
def scoreSentences (embeddings):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        x = sess.run(embeddings)
    return x

def saveWeights(elmoFN, x):
    with open('./models/'+elmoFN,'wb') as f: pkl.dump(x, f)
    return

In [32]:
import math 

batch_size = 50                    
numPlots = len(sentences)
numfiles = math.ceil(numPlots/batch_size) 
print('No of expected Elmo Weigh files : ', str(numfiles))

start = 0
end = batch_size
for file in range(1,numfiles+1):
    elmoFN = 'elmoPlotsweights' + str(start) + 'to' + str(end) + '.pkl'
    print('Score sentences:', elmoFN)

        # Define Embeddings, Score, and Store Weights one Batch of N at time
    embeddings = defineEmbeddings(start, end)
    x = scoreSentences(embeddings)
    saveWeights(elmoFN,x)
    start = end
    end = end+batch_size

No of expected Elmo Weigh files :  25
Score sentences: elmoPlotsweights0to50.pkl
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Score sentences: elmoPlotsweights50to100.pkl
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Score sentences: elmoPlotsweights100to150.pkl
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Score sentences: elmoPlotsweights1150to1200.pkl
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Score sentences: elmoPlotsweights1200to1250.pkl
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [39]:

import pickle as pkl
import numpy as np
import glob

def readElmoWeightFile(file):
    with open(elmodir+file, 'rb') as f:
        elmo_file = pkl.load(f)
    return list(elmo_file)
  
def readWeights(elmodir, modelname):
    noFiles = len([name for name in os.listdir(elmodir) if modelname in name ])
    xin = readElmoWeightFile('elmoPlotsweights0to'+str(batch_size)+'.pkl')
    size = batch_size
    for x in range(1, noFiles):
        start = x*size
        end = start+size
        filename = 'elmoPlotsweights' + str(start) + 'to' + str(end) + '.pkl'
        print(filename)
        xin += readElmoWeightFile(filename)
    return xin

In [42]:
!ls models/

actorGenreDict.pkl		elmoPlotsweights300to350.pkl
actorGenreVect.pkl		elmoPlotsweights350to400.pkl
bio_dict.pkl			elmoPlotsweights400to450.pkl
celebrityBio.json		elmoPlotsweights450to500.pkl
celebrityLookup.pkl		elmoPlotsweights500to550.pkl
elmoBioweights0to100.pkl	elmoPlotsweights50to100.pkl
elmoBioweights.pkl		elmoPlotsweights550to600.pkl
elmoPlotsweights0to50.pkl	elmoPlotsweights600to650.pkl
elmoPlotsweights1000to1050.pkl	elmoPlotsweights650to700.pkl
elmoPlotsweights100to150.pkl	elmoPlotsweights700to750.pkl
elmoPlotsweights1050to1100.pkl	elmoPlotsweights750to800.pkl
elmoPlotsweights1100to1150.pkl	elmoPlotsweights800to850.pkl
elmoPlotsweights1150to1200.pkl	elmoPlotsweights850to900.pkl
elmoPlotsweights1200to1250.pkl	elmoPlotsweights900to950.pkl
elmoPlotsweights150to200.pkl	elmoPlotsweights950to1000.pkl
elmoPlotsweights200to250.pkl	genreStats.pkl
elmoPlotsweights250to300.pkl


In [41]:
#Export Dictonary and GenrePlot Vectors
import pickle
xin = readWeights(elmodir, 'Plot')
print("Elmo Vector Shape {0}, {1}".format( len(xin), len(xin[1])) )

pickle.dump( actorGenreJson, open( "./models/actorGenreDict.pkl", "wb" ) )
pickle.dump( xin, open( "./models/actorGenreVect.pkl", "wb" ) )


elmoPlotsweights50to100.pkl
elmoPlotsweights100to150.pkl
elmoPlotsweights150to200.pkl
elmoPlotsweights200to250.pkl
elmoPlotsweights250to300.pkl
elmoPlotsweights300to350.pkl
elmoPlotsweights350to400.pkl
elmoPlotsweights400to450.pkl
elmoPlotsweights450to500.pkl
elmoPlotsweights500to550.pkl
elmoPlotsweights550to600.pkl
elmoPlotsweights600to650.pkl
elmoPlotsweights650to700.pkl
elmoPlotsweights700to750.pkl
elmoPlotsweights750to800.pkl
elmoPlotsweights800to850.pkl
elmoPlotsweights850to900.pkl
elmoPlotsweights900to950.pkl
elmoPlotsweights950to1000.pkl
elmoPlotsweights1000to1050.pkl
elmoPlotsweights1050to1100.pkl
elmoPlotsweights1100to1150.pkl
elmoPlotsweights1150to1200.pkl
elmoPlotsweights1200to1250.pkl
Elmo Vector Shape 1228, 1024


In [45]:
# Copy models up to S3 bucket
bucket = os.popen('aws ssm get-parameter --name jupyterBucketName-' + PLATFORM + ' --with-decryption | jq -r .Parameter.Value').read()
print(bucket)
executable = 'aws s3 sync ./models/ s3://' + bucket.strip() + '/models/ '
print(executable)
myCmd = os.popen(executable).read()
print(myCmd)

bdso-celebrity-content-dev-vpc-05447d97de18e28f8

aws s3 sync ./models/ s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/models/ 
Completed 256.0 KiB/16.7 MiB (2.2 MiB/s) with 22 file(s) remaining
Completed 512.0 KiB/16.7 MiB (4.1 MiB/s) with 22 file(s) remaining
Completed 657.0 KiB/16.7 MiB (5.1 MiB/s) with 22 file(s) remaining
Completed 913.0 KiB/16.7 MiB (6.3 MiB/s) with 22 file(s) remaining
Completed 925.7 KiB/16.7 MiB (6.3 MiB/s) with 22 file(s) remaining
upload: models/celebrityBio.json to s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/models/celebrityBio.json
Completed 925.7 KiB/16.7 MiB (6.3 MiB/s) with 21 file(s) remaining
Completed 1.2 MiB/16.7 MiB (7.8 MiB/s) with 21 file(s) remaining  
Completed 1.4 MiB/16.7 MiB (9.2 MiB/s) with 21 file(s) remaining  
Completed 1.6 MiB/16.7 MiB (10.3 MiB/s) with 21 file(s) remaining 
upload: models/elmoPlotsweights1000to1050.pkl to s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/models/elmoPlotsweights1000to1050.pkl
Comple

# Push Model to Production

In [4]:
import pickle

actorGenreJson = pickle.load( open( './models/actorGenreDict.pkl', 'rb' ) )
xin = pickle.load( open( './models/actorGenreVect.pkl', 'rb' ) )


## Test Elasticsearch to store the vertor and peform cosine similarity score.


In [ ]:
import requests
from pprint import pprint
from datetime import datetime
from elasticsearch import Elasticsearch
import json


_index='genresplot'
esIndex = 'http://'+ESHOST+':'+ESPORT+'/'+_index  
es=Elasticsearch([{'host':ESHOST,'port':ESPORT}])

_doc= 'genreplot'

headers={"Content-Type": "application/json"}

In [ ]:
request_body = {
  "mappings": {
    "properties": {
      "_actorID" : {
        "type" : "keyword"
      },     
      "celebrity" : {
        "type" : "keyword"
      }, 
      "genre" : {
        "type" : "keyword"
      }, 
                
      "genre_vector": {
        "type": "dense_vector",
        "dims": 1024
      }

    }
  }
}

try:
    es.indices.delete(index = _index)
except:
    pass
    
es.indices.create(index = _index, body = request_body)

In [ ]:
actorGenreJson[0]

In [ ]:
# Load Facail data into ES
for i in range(len(actorGenreJson)):
    actorGenre =actorGenreJson[i]
    _id = actorGenre['_actorID']+'-'+actorGenre['genre']
    #print(_id)
    doc = {
        "_actorID" : actorGenre['_actorID'],
        "celebrity": actorGenre['actor'],
        "genre": actorGenre['genre'],
        "genre_vector": [float(x) for x in xin[i]]
        }
    r = requests.put(esIndex+'/_doc/'+_id, headers= headers, data = json.dumps(doc))
    print (r.text)

In [ ]:
def getCelbGenre( celebrity): 
    q={
      "_source": {
        "includes": [ "genre"]
        },
      "query": { 
        "bool": { 
          "must": [
            { "match": { "celebrity":celebrity}}
          ]
        }
      }
    }
    res= es.search(index=_index, body=json.dumps(q))
    if res['hits']['hits']:
        results=[]
        matches = res['hits']['hits']
        for match in matches:
            results.append(match['_source']['genre'])
        return results
    else:
        return None
            


In [ ]:
def getCelbGenreVec( celebrity, genre): 
    q={
      "query": { 
        "bool": { 
          "must": [
            { "match": { "celebrity":celebrity}}
          ],
          "filter": [ 
            { "term":  { "genre": genre }}
          ]
        }
      }
    }
    res= es.search(index=_index, body=json.dumps(q))
    if res['hits']['hits']:
        match = res['hits']['hits'][0]
        #print(match)
        return match['_source']['genre_vector']
    else:
        return None
        



In [ ]:
def getSimilarGenreVec( genre_vec, genre): 
    q={
          "query": {
            "script_score": {
             "query": {
                    "terms" : { "genre" : [genre]}
                },
              "script": {
                "source": "cosineSimilarity(params.query_vector, doc['genre_vector']) + 1.0", 
                "params": {
                  "query_vector": genre_vec
                }
              }
            }
          }
        }
    #print(json.dumps(q))
    res= es.search(index=_index, body=q)

    res= es.search(index=_index, body=json.dumps(q))
    if res['hits']['hits']:
        matches = res['hits']['hits']
        #print(match)
        results=[]
        for match in matches:
            results.append( { 'genre': genre, 'celebrity': match['_source']['celebrity'], 'score': match['_score']})
        return (results) 
    else:
        return None

    



In [ ]:
# 'Comedy, Animation, Action, Documentary, Short, Adventure, Drama, Biography, Music, Thriller, Horror, Mystery, Crime, Sport, Musical, , Family, Game-Show, Reality-TV, Fantasy, Romance, Sci-Fi, History, War, Western, News, Film-Noir'
celebritySearch = 'Robert Downey Jr.'
genreSearch='Adventure'

getCelbGenre(celebritySearch)


In [ ]:
genreVec = getCelbGenreVec(celebritySearch, genreSearch)
getSimilarGenreVec( genreVec, genreSearch)

# Recommendations Celebrity alterative based on their past genre movies.

* For a given Celebrity, reference all the genre we have information on. Retrieve the genre vector space based on 
the list of movies plots the actor performed in the past.
* Calculate the cosine distince and recommend other celebrities who performed in similar movies based on their plots 

In [ ]:
resByDrama=[]

for genreSearch in getCelbGenre(celebritySearch):
    genreVec = getCelbGenreVec(celebritySearch, genreSearch)
    genreRecs = getSimilarGenreVec( genreVec, genreSearch)
    # Remove the search celebrirty from the recomendations
    resByDrama = resByDrama+ [i  for i in genreRecs if i['celebrity']!= celebritySearch ] 
    
resByDrama

In [ ]:
from IPython.display import Image, HTML, display

display(HTML("<h2>Get similar celebrities for: %s</h2>"% celebritySearch))

pd.DataFrame.from_dict(resByDrama, dtype=str)   

In [ ]:

resultDF = pd.DataFrame.from_dict(resByDrama, dtype=str)   

resultDF.info()

In [ ]:
# Data Visualize Cluster

In [ ]:
from sklearn.cluster import DBSCAN, KMeans
import numpy as np
import matplotlib.pyplot as plt

X = np.array([float(x) for x in xin[i]])

clust = KMeans().fit(X)
y_kmeans = KMeans.predict(X)